# Example 1: Converting Models to OpenVINO IR

In this example, you will learn about converting models to an OpenVINO Immediate Representation (IR). This representation will allow OpenVINO to conduct inference optimally on Intel hardware.

### Sections
1.1 **Convert Keras Models (.hdf5):** We will convert a fault segmentation Keras model to a frozen graph.<br/>
1.2 **Convert Tensorflow Frozen Graphs (.pb):** Using our newly converted frozen graph, we will use the model optimizer to convert from frozen graph to IR. <br/>
1.3 **Convert ONNX Models (.onnx):** We will convert a salt identification Pytorch model to its ONNX equivalent and then convert to IR.<br/>

### Imports

Below are some important imports and global variables that we will need to set up before moving forward. Make sure to run `pip install -r requirements.txt` and build/pull the docker image for Open Seismic.

In [1]:
import os
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
import warnings

import numpy as np
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras.layers import *
from keras.models import load_model
import tensorflow.keras.backend as K
from assets.example1_assets.unet3 import cross_entropy_balanced
import shutil, sys

import torch
from assets.example1_assets.texture_net import TextureNet

from pathlib import PurePath
assets_path = PurePath('assets', 'example1_assets')
open_seismic_docker = 'open_seismic'

Using TensorFlow backend.


## Section 1.1: Convert Keras Models (.hdf5)

OpenVINO supports popular deep learning frameworks, and [here](https://docs.openvinotoolkit.org/latest/openvino_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) is a list of the frameworks that it supports. For Keras models, we will first convert to a Tensorflow frozen graph representation. In section 1.2, we will convert the Tensorflow frozen graph representation into OpenVINO's Immediate Representation (IR). Below, we will walk through how to convert a Keras model to a frozen graph.

### Steps
1. Using the Keras library, load the model into memory by specifying the path to the Keras model as well as custom objects that you defined in your network.

In [2]:
path_to_keras = str(assets_path.joinpath('fseg-60.hdf5'))
model = load_model(path_to_keras, custom_objects={'cross_entropy_balanced': cross_entropy_balanced})
frozen_model_path = str(assets_path)
frozen_model_name = 'fseg-60.pb'

2. Get the node names that exist within the Keras model layers

In [3]:
tf.get_logger().setLevel('INFO')
K.set_image_data_format('channels_last')

output_node_names = [node.op.name for node in model.outputs]

3. Instantiate a Keras session and use the graph_util function from Tensorflow's framework module to create a constant graph.

In [4]:
sess = K.get_session()
constant_graph = graph_util.convert_variables_to_constants(sess,
                                                           sess.graph.as_graph_def(),
                                                           output_node_names)

INFO:tensorflow:Froze 30 variables.
INFO:tensorflow:Converted 30 variables to const ops.


4. Finally, write the constant graph to disk with name `frozen_model_name` at `frozen_model_path`.

In [5]:
graph_io.write_graph(constant_graph, frozen_model_path, frozen_model_name, as_text=False)

'assets/example1_assets/fseg-60.pb'

In four easy steps, we were able to convert a Keras model into a frozen graph.

## Section 1.2: Convert Tensorflow Frozen Graphs (.pb)

In the previous section, we walked through how to convert a Keras model into its frozen graph equivalent. This was a necessary step because we can now convert our frozen graph into an OpenVINO IR. Below, we will walk through how to convert a frozen graph model to OpenVINO IR using the Open Seismic Docker image.

### Steps
1. Familiarize yourself with the Model Optimizer options. For a detailed explanation, go [here](https://docs.openvinotoolkit.org/latest/openvino_docs_MO_DG_prepare_model_Config_Model_Optimizer.html).

In [6]:
model_optimizer_cmd = f"""
docker run {open_seismic_docker} /bin/bash executables/mo.sh -h
"""
! {model_optimizer_cmd}

usage: mo.py [options]

optional arguments:
  -h, --help            show this help message and exit
  --framework {tf,caffe,mxnet,kaldi,onnx}
                        Name of the framework used to train the input model.

Framework-agnostic parameters:
  --input_model INPUT_MODEL, -w INPUT_MODEL, -m INPUT_MODEL
                        Tensorflow*: a file with a pre-trained model (binary
                        or text .pb file after freezing). Caffe*: a model
                        proto file with model weights
  --model_name MODEL_NAME, -n MODEL_NAME
                        Model_name parameter passed to the final create_ir
                        transform. This parameter is used to name a network in
                        a generated IR and output .xml/.bin files.
  --output_dir OUTPUT_DIR, -o OUTPUT_DIR
                        Directory that stores the generated IR. By default, it
                        is the directory from where the Model Optimizer is
                        lau

2. Specify the appropriate configurations.

In [7]:
# Model expects a 5-dimensional input
model.input.get_shape().as_list()

[None, None, None, None, 1]

In [8]:
phys_mnt_vol = str(PurePath(os.getcwd()).joinpath(assets_path))
docker_mnt_vol = '/mnt_vol'
docker_frozen_model_path = f'{docker_mnt_vol}/fseg-60.pb'
input_shape = str([1] + [128] * 3 + [1]).replace(" ", "")
data_type = 'FP32'
docker_output_model_path = f'{docker_mnt_vol}/IR_fault/'
ov_model_name = 'fseg-60'
configs = f"--input_model {docker_frozen_model_path} \
            --input_shape {input_shape} \
            --data_type {data_type} \
            --output_dir {docker_output_model_path}  \
            --model_name {ov_model_name}"

model_optimizer_cmd = f"docker run -v {phys_mnt_vol}:{docker_mnt_vol} {open_seismic_docker} /bin/bash executables/mo.sh {configs}"
model_optimizer_cmd # Checking the command to see if this works

'docker run -v /home/akhorkin/Repositories/open_seismic/examples/assets/example1_assets:/mnt_vol open_seismic /bin/bash executables/mo.sh --input_model /mnt_vol/fseg-60.pb             --input_shape [1,128,128,128,1]             --data_type FP32             --output_dir /mnt_vol/IR_fault/              --model_name fseg-60'

3. Run the model optimizer on Open Seismic!

In [9]:
! {model_optimizer_cmd}

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/mnt_vol/fseg-60.pb
	- Path for generated IR: 	/mnt_vol/IR_fault/
	- IR output name: 	fseg-60
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	[1,128,128,128,1]
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	None
	- Reverse input channels: 	False
TensorFlow specific parameters:
	- Input model in text protobuf format: 	False
	- Path to model dump for TensorBoard: 	None
	- List of shared libraries with TensorFlow custom layers implementation: 	None
	- Update the configuration file with input/output node names: 	None
	- Use configuration file used to generate the model with Ob

Congratulations! You have successfully converted a Keras model to a Tensorflow frozen graph and optimized it to an OpenVINO IR! In the next section, we will cover how to do the same thing but with a Pytorch model and ONNX.

## Section 1.3: Convert ONNX Models (.onnx)

In this section, we will convert a Pytorch model to its ONNX equivalent. Then, we will use the model optimizer to get an OpenVINO IR. Below, we will walk through the steps.

### Steps
1. Convert the Pytorch model to its ONNX equivalent. The weights are provided for ease-of-use.

In [10]:
pt_filename = str(assets_path.joinpath('salt.pt'))
device = torch.device('cpu')
network = TextureNet(n_classes=2)
network.load_state_dict(torch.load(pt_filename, map_location=device))
network.eval()

output_filename = str(assets_path.joinpath('salt.onnx'))
input_var = torch.randn(1, 1, 65, 65, 65)
torch.onnx.export(network, input_var, output_filename, export_params=True)

2. Specify the appropriate configurations for the model optimizer

In [11]:
phys_mnt_vol = str(PurePath(os.getcwd()).joinpath(assets_path))
docker_mnt_vol = '/mnt_vol'
docker_onnx_model_path = f'{docker_mnt_vol}/salt.onnx'
input_shape = str([1] * 2 + [65] * 3).replace(" ", "")
data_type = 'FP32'
docker_output_model_path = f'{docker_mnt_vol}/IR_salt/'
ov_model_name = 'salt'
configs = f"--input_model {docker_onnx_model_path} \
            --input_shape {input_shape} \
            --data_type {data_type} \
            --output_dir {docker_output_model_path}  \
            --model_name {ov_model_name}"

model_optimizer_cmd = f"docker run -v {phys_mnt_vol}:{docker_mnt_vol} {open_seismic_docker} /bin/bash executables/mo.sh {configs}"
model_optimizer_cmd # Checking the command to see if this works

'docker run -v /home/akhorkin/Repositories/open_seismic/examples/assets/example1_assets:/mnt_vol open_seismic /bin/bash executables/mo.sh --input_model /mnt_vol/salt.onnx             --input_shape [1,1,65,65,65]             --data_type FP32             --output_dir /mnt_vol/IR_salt/              --model_name salt'

3. Run the model optimizer in Open Seismic!

In [12]:
! {model_optimizer_cmd}

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/mnt_vol/salt.onnx
	- Path for generated IR: 	/mnt_vol/IR_salt/
	- IR output name: 	salt
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	[1,1,65,65,65]
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	None
	- Reverse input channels: 	False
ONNX specific parameters:
Model Optimizer version: 	2021.1.0-1237-bece22ac675-releases/2021/1

[ SUCCESS ] Generated IR version 10 model.
[ SUCCESS ] XML file: /mnt_vol/IR_salt/salt.xml
[ SUCCESS ] BIN file: /mnt_vol/IR_salt/salt.bin
[ SUCCESS ] Total execution time: 2.87 seconds. 
[ SUCCESS ] Memory consumed: 235 MB. 
It's been a while, c

Congratulations! You have successfully converted a Pytorch model to an ONNX model and optimized it to an OpenVINO IR!

## Summary

In this example, we learned:
1. How to convert Keras models to Tensorflow frozen graphs
2. How to optimize Tensorflow frozen graphs to IR
3. How to convert Pytorch models to ONNX and optimize to IR